In [1]:
import numpy as np
import keras
from keras import callbacks
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
from math import floor, log10
def dig(number, dig=2):
    return round(number, dig - floor(log10(abs(number))) - 1)

In [3]:
img_row = 108
img_col = 108
path = "E:/database108/"
batch_size = 64
output_classes = 24

In [4]:
import random
import pandas as pd
x_train_ori = []
y_train_ori = []
x_test_ori = []
y_test_ori = []
for i in range(516):
    print(i)
    file = pd.read_csv(path + "database_{}.csv".format(i), dtype=str)
    array = file.values
    array = list(array)
    train_test = random.sample(range(100), 25)
    
    for j in train_test[0:20]:
        sample = list(array[j])
        lst = []
        for x in sample[1:11665]:
            if type(x) == str:
                x = "".join(x.split())
                lst.append(float(x))
            elif type(x) == float:
                lst.append(x)
            else:
                print(type(x))
        x_train_ori.append(lst)
        y_train_ori.append(sample[0])
        
    for l in train_test[20:]:
        sample = list(array[l])
        lst = []
        for x in sample[1:11665]:
            if type(x) == str:
                x = "".join(x.split())
                lst.append(float(x))
            elif type(x) == float:
                lst.append(x)
            else:
                print(type(x))
        x_test_ori.append(lst)
        y_test_ori.append(sample[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
print(np.shape(x_train_ori))
print(np.shape(x_test_ori))
print(np.shape(y_train_ori))
print(np.shape(y_test_ori))

(10320, 11664)
(2580, 11664)
(10320,)
(2580,)


In [6]:
def y_convert(array, flag=0):
    y = []
    for i in array: # i is the name of each sample:
        name = i.split("_")
        name = name[1:5]
        if flag == 0:
            # category of appearance of a note
            category = np.zeros(24, dtype=int)
            for j in name:
                if j != "x":
                    num = int(j)
                    category[num-60] = 1
            y.append(list(category))
    return y

In [7]:
test = ["C_60_61_65_x", "C_67_76_78_79"]
print(y_convert(test))

[[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0]]


In [8]:
lst_1 = []
lst_2 = []

for i in y_train_ori:
    label = "".join(i.split())
    lst_1.append(label)
for j in y_test_ori:
    label = "".join(j.split())
    lst_2.append(label)
    
y_train_ori = y_convert(lst_1)
y_test_ori = y_convert(lst_2)

In [11]:
y_train_ori = np.array(y_train_ori)
y_test_ori = np.array(y_test_ori)
x_train_ori = np.array(x_train_ori)
x_test_ori = np.array(x_test_ori)

print(y_train_ori.shape)
print(y_test_ori.shape)

(10320, 24)
(2580, 24)


In [13]:
no_training = 10320
no_test = 2580
x_train_ori = np.array(x_train_ori).flatten()
x_train_ori = x_train_ori.reshape(no_training, img_row, img_col, 1)
x_test_ori = np.array(x_test_ori).flatten()
x_test_ori = x_test_ori.reshape(no_test, img_row, img_col, 1)

In [14]:
print(x_train_ori.shape)
input_shape = (108, 108, 1)

(10320, 108, 108, 1)


In [17]:
def build_model_A ():    
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.4))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])
    return model

In [18]:
# randomly taking 10320 data samples
classifier_A = build_model_A()
hist_A = classifier_A.fit(x_train_ori, y_train_ori, batch_size=128, nb_epoch=50, verbose=1, 
                        validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 10320 samples, validate on 2580 samples
Epoch 1/50
10320/10320 [==============================] - 7s 675us/step - loss: 9.4651 - acc: 0.3154 - val_loss: 8.2796 - val_acc: 0.4023
Epoch 2/50
10320/10320 [==============================] - 6s 604us/step - loss: 7.1168 - acc: 0.3332 - val_loss: 6.7068 - val_acc: 0.2857
Epoch 3/50
10320/10320 [==============================] - 6s 602us/step - loss: 6.5351 - acc: 0.2939 - val_loss: 6.3834 - val_acc: 0.1795
Epoch 4/50
10320/10320 [==============================] - 6s 605us/step - loss: 6.3285 - acc: 0.2922 - val_loss: 6.0262 - val_acc: 0.2694
Epoch 5/50
10320/10320 [==============================] - 6s 606us/step - loss: 6.2187 - acc: 0.2841 - val_loss: 5.7460 - val_acc: 0.2322
Epoch 6/50
10320/10320 [==============================] - 6s 601us/step - loss: 6.1511 - acc: 0.2830 - val_loss: 5.6148 - val_acc: 0.2353
Epoch 7/50
10320/10320 [==============================] - 6s 607us/step - loss: 6.0970 - acc: 0.2807 - val_loss: 5.6133 - v

In [25]:
fname = "weight-10320-CNN_A.hdf5"
classifier_A.save_weights(fname, overwrite=True)

In [21]:
def build_model_B():
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #forth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.4))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(128, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
        
    #second FC layer
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [22]:
# randomly taking 10320 data samples
classifier_B = build_model_B()
hist_B = classifier_B.fit(x_train_ori, y_train_ori, batch_size=64, nb_epoch=50, verbose=1, 
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 10320 samples, validate on 2580 samples
Epoch 1/50
10320/10320 [==============================] - 9s 843us/step - loss: 7.6112 - acc: 0.2736 - val_loss: 5.9686 - val_acc: 0.1477
Epoch 2/50
10320/10320 [==============================] - 7s 698us/step - loss: 6.1199 - acc: 0.2801 - val_loss: 5.6523 - val_acc: 0.1969
Epoch 3/50
10320/10320 [==============================] - 7s 692us/step - loss: 5.8392 - acc: 0.2703 - val_loss: 5.4194 - val_acc: 0.3198
Epoch 4/50
10320/10320 [==============================] - 7s 693us/step - loss: 5.6918 - acc: 0.2750 - val_loss: 5.3505 - val_acc: 0.2399
Epoch 5/50
10320/10320 [==============================] - 7s 694us/step - loss: 5.6087 - acc: 0.2697 - val_loss: 5.3014 - val_acc: 0.3360
Epoch 6/50
10320/10320 [==============================] - 7s 694us/step - loss: 5.5397 - acc: 0.2706 - val_loss: 5.2748 - val_acc: 0.3163
Epoch 7/50
10320/10320 [==============================] - 7s 693us/step - loss: 5.4848 - acc: 0.2698 - val_loss: 5.2431 - v

In [26]:
fname = "weight-10320-CNN_B.hdf5"
classifier_B.save_weights(fname, overwrite=True)

In [23]:
def build_model_C():
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])
    return model

In [24]:
# randomly taking 10320 data samples
classifier_C = build_model_C()
hist_C = classifier_C.fit(x_train_ori, y_train_ori, batch_size=64, nb_epoch=50, verbose=1,
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 10320 samples, validate on 2580 samples
Epoch 1/50
10320/10320 [==============================] - 7s 695us/step - loss: 7.5317 - acc: 0.3270 - val_loss: 6.6098 - val_acc: 0.1229
Epoch 2/50
10320/10320 [==============================] - 6s 623us/step - loss: 5.9865 - acc: 0.2791 - val_loss: 6.0474 - val_acc: 0.1023
Epoch 3/50
10320/10320 [==============================] - 6s 623us/step - loss: 5.8112 - acc: 0.2726 - val_loss: 5.6609 - val_acc: 0.2047
Epoch 4/50
10320/10320 [==============================] - 6s 623us/step - loss: 5.7467 - acc: 0.2747 - val_loss: 5.4659 - val_acc: 0.2950
Epoch 5/50
10320/10320 [==============================] - 6s 623us/step - loss: 5.7031 - acc: 0.2665 - val_loss: 5.7294 - val_acc: 0.1845
Epoch 6/50
10320/10320 [==============================] - 6s 623us/step - loss: 5.6828 - acc: 0.2770 - val_loss: 5.5264 - val_acc: 0.3174
Epoch 7/50
10320/10320 [==============================] - 6s 623us/step - loss: 5.6523 - acc: 0.2676 - val_loss: 5.4765 - v

In [27]:
fname = "weight-10320-CNN_C.hdf5"
classifier_C.save_weights(fname, overwrite=True)